## Noteable Plugin

```bash
pip install chatlab[noteable]
```

ChatLab can optionally be installed with a Noteable `NotebookClient`. Using this, you can re-create the Noteable Plugin experience in your own Jupyter Notebook environment. The `NotebookClient` allows you to pass only the functionsw you want, allowing you to tailor how the LLM responds. There's also a significant speed improvement over the Noteable Plugin because:

- The `NotebookClient` maintains a copy of the realtime notebook, allowing for faster cell creation and retrieval
- The one notebook per conversation model allows for faster LLM operations since it doesn't have to "type out" file IDs and project IDs continuously

You can even use this to create your own isolated version of the code interpreter. It's up to you to mix and match the functions you want to expose to the LLM.


In [1]:
from chatlab import models, Chat, FunctionRegistry, system
from chatlab.builtins.noteable import NotebookClient

In order to get the closest experience to the plugin, we'll grab the "plugin prompt" that comes in the Noteable manifest for the ChatGPT Plugin. We can then provide this in a `system` prompt for the model.


In [2]:
import requests

plugin_prompt = requests.get("https://chat.noteable.io/.well-known/ai-plugin.json").json()['description_for_model']

print(plugin_prompt[:90].strip() + "...")

On https://app.noteable.io, create and run Jupyter notebooks with code, markdown, and SQL...


In [3]:
import os

registry = FunctionRegistry()


async def create_notebook(file_name: str):
    """Create a notebook to use in this conversation"""
    nc = await NotebookClient.create(file_name=file_name, token=os.environ.get("NOTEABLE_TOKEN"))

    # Register all the regular notebook operations
    registry.register_functions(nc.chat_functions)
    # Let the model do `python` (which creates and runs a cell)
    registry.python_hallucination_function = nc.python

    return f"Notebook created at {nc.notebook_url}"


registry.register(create_notebook)

{'name': 'create_notebook',
 'description': 'Create a notebook to use in this conversation',
 'parameters': {'type': 'object',
  'properties': {'file_name': {'type': 'string'}},
  'required': ['file_name']}}

In [4]:
chat = Chat(system(plugin_prompt), model=models.GPT_3_5_TURBO_16K_0613, function_registry=registry)
chat.function_registry.api_manifest()

{'functions': [{'name': 'create_notebook',
   'description': 'Create a notebook to use in this conversation',
   'parameters': {'type': 'object',
    'properties': {'file_name': {'type': 'string'}},
    'required': ['file_name']}}],
 'function_call': 'auto'}

In [5]:
await chat(
    "Let's make a notebook to analyze the Berkeley 311 calls from https://data.cityofberkeley.info/resource/bscu-qpbu.json. Please run code, perform analysis, and do the full EDA lifecycle for me. I recommend running the cells yourself as I want you to make sure to work based on the actual data format. Show me your best data scientist."
)

Received un-modeled RTU message msg.channel='kernels/notebook-kernel-6d0ff2eca86d4aec898c' msg.event='variable_explorer_update_request'


Here are the results of the analysis of the Berkeley 311 calls dataset:

1. DataFrame Summary: The dataset contains 23 columns with various data types. Some columns have missing values.

2. Data Cleaning: The column names have been converted to lowercase. The date columns have been converted to datetime format.

3. Data Analysis:
   - Distribution of Request Categories: This bar plot shows the distribution of request categories in the dataset.
   - Distribution of Request Subcategories: This bar plot shows the distribution of request subcategories in the dataset.
   - Request Status: This bar plot shows the distribution of request status in the dataset.
   - Time Series Analysis: This line plot shows the time series of request counts over time.

You can view the full analysis in the [notebook](https://app.noteable.io/f/6d0ff2ec-a86d-4aec-898c-fe51a04b7b6a).

Is there anything specific you would like to explore further or any additional analysis you would like me to perform?

In [6]:
await chat("Please add some narrative markdown cells to the notebook too.")

I have added narrative markdown cells to the notebook. The notebook now includes the following sections:

1. Exploratory Data Analysis (EDA): Provides an overview of the notebook and the dataset.

2. Data Import and Exploration: Demonstrates the process of importing the necessary libraries and loading the dataset.

3. Data Cleaning: Covers the steps involved in cleaning the dataset, such as column name cleanup and date column conversion.

4. Data Analysis: Includes visualizations and analysis of the dataset, including the distribution of request categories, distribution of request subcategories, request status distribution, and time series analysis of request counts over time.

You can access the full notebook [here](https://app.noteable.io/f/6d0ff2ec-a86d-4aec-898c-fe51a04b7b6a). Let me know if there's anything else you would like to add or if you have any specific analysis requests!

In [7]:
chat.function_registry.api_manifest()

{'functions': [{'name': 'create_notebook',
   'description': 'Create a notebook to use in this conversation',
   'parameters': {'type': 'object',
    'properties': {'file_name': {'type': 'string'}},
    'required': ['file_name']}},
  {'name': 'create_cell',
   'description': 'Create a code, markdown, or SQL cell.',
   'parameters': {'type': 'object',
    'properties': {'source': {'type': 'string'},
     'cell_type': {'type': 'string',
      'enum': ('code', 'markdown', 'sql'),
      'default': 'code'},
     'cell_id': {'type': 'string', 'default': None},
     'and_run': {'type': 'boolean', 'default': False},
     'after_cell_id': {'type': 'string', 'default': None},
     'db_connection': {'type': 'string', 'default': None},
     'assign_results_to': {'type': 'string', 'default': None}},
    'required': ['source']}},
  {'name': 'run_cell',
   'description': 'Run a Cell within a Notebook by ID.',
   'parameters': {'type': 'object',
    'properties': {'cell_id': {'type': 'string'}},
    '